In [1]:
import arcpy
import pandas as pd
import psycopg2
import requests
from shapely.wkb import loads as wkb_loads
import requests
import csv
from arcpy import env
import os
import numpy as np
import io
import json
from io import StringIO
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
import zipfile
from osgeo import gdal
import geopandas
import pyproj
import random
from sqlalchemy import create_engine

Note: Portions of this notebook were developed using AI generated code.

### Interpolation

In [3]:
# We will connect to the original database
db_params = {
    'database': 'lab1.2',  
    'user': '(USER)',  
    'password': '(PASSWORD), 
    'host': '(HOST)', 
    'port': '5432' 
}

conn = psycopg2.connect(**db_params)
cursor = conn.cursor()

# Then  we will select the needed table
sql_query = "SELECT grid_code, wkt_geom FROM minn_elevation_wkt"
cursor.execute(sql_query)
data = cursor.fetchall()

# Then convert fetched data to a dataframe
elevation_df = pd.DataFrame(data, columns=['grid_code', 'wkt_geom'])


conn.close()


In [ ]:
connection_properties = {
    'database': 'lab3',
    'user': '(USER)',  
    'password': '(PASSWORD), 
    'host': '(HOST)',
    'port': '5432'
}


db_connection_file = r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5572 Lab 2\database_connection.sde"
arcpy.CreateDatabaseConnection_management(out_folder_path=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5572 Lab 2",
                                          out_name="database_connection",
                                          database_platform="POSTGRESQL",
                                          instance=connection_properties['host'],
                                          account_authentication="DATABASE_AUTH",
                                          username=connection_properties['user'],
                                          password=connection_properties['password'],
                                          save_user_pass="SAVE_USERNAME",
                                          database=connection_properties['database'],
                                          version_type="TRANSACTIONAL",
                                          version="dbo.DEFAULT"
                                          )


In [5]:
# We will then create our elevation feature class
output_fc = "Elevation_Data"

# We will define the spatial reference for UTM Zone 15N and our desired deature class
utm_spatial_reference = arcpy.SpatialReference(32615)
wgs84_spatial_reference = arcpy.SpatialReference(4326)  # EPSG code for WGS84


arcpy.management.CreateFeatureclass(arcpy.env.workspace, output_fc, "POINT", spatial_reference=wgs84_spatial_reference)
arcpy.management.AddField(output_fc, "grid_code", "FLOAT")

# Then we iterate over each row in the elevation dataframe
with arcpy.da.InsertCursor(output_fc, ["SHAPE@", "grid_code"]) as cursor:
    for index, row in elevation_df.iterrows():
        wkt_geom = row['wkt_geom']
        grid_code = row['grid_code']
        x, y = map(float, wkt_geom[7:-1].split())
        
        # Then we can create point geometry from X and Y coordinates
        point = arcpy.Point(x, y)
        geom = arcpy.PointGeometry(point, utm_spatial_reference)
        
        # We then project point geometry to WGS84 (latitude and longitude)
        projected_geom = geom.projectAs(wgs84_spatial_reference)
        
        # Then we insert each row into the feature class
        cursor.insertRow((projected_geom, grid_code))

In [7]:
# Next we need to sample the larger raster data
input_feature_layer = "Elevation_Data"
output_feature_layer = "Sampled_Elevation_Data"

total_features = int(arcpy.GetCount_management(input_feature_layer).getOutput(0))

# We will select 1000 indices without replacement
sample_indices = random.sample(range(total_features), 1000)

# Then we will create a SQL query to select the sampled features
sql_query = f"OBJECTID IN ({', '.join(map(str, sample_indices))})"

# We can then create the output feature layer with sampled features
arcpy.management.SelectLayerByAttribute(input_feature_layer, "NEW_SELECTION", sql_query)
arcpy.management.CopyFeatures(input_feature_layer, output_feature_layer)

print("Sample data collected)")

<Result 'C:\\Users\\conno\\OneDrive\\Documents\\ArcGIS\\Projects\\Lab 3 v2\\Lab 3 v2.gdb\\Sampled_Elevation_Data'>

In [ ]:
# In order to test our interpolations later, we will need to create another subset of this data and remove it from the feature layer
input_feature_layer = "Sampled_Elevation_Data"

# We will save it as a separate feature layer
output_feature_layer = "Random_Selected_Elevation_Features"
output_feature_class = "Random_Selected_Elevation_Features_Class"

# We first need to get the total count of features in the input feature layer
total_features_count = int(arcpy.GetCount_management(input_feature_layer).getOutput(0))

# Then I will generate a list of 100 random indices
random_indices = random.sample(range(1, total_features_count + 1), 100)

# I will use a SQL expression to select the randomly chosen features
sql_expression = "OBJECTID IN ({})".format(','.join(map(str, random_indices)))

# Then I will create a new feature layer with the randomly selected features
arcpy.MakeFeatureLayer_management(input_feature_layer, output_feature_layer, sql_expression)

print("Randomly selected 100 features and created a new feature layer:", output_feature_layer)

# The I will save the selected features to a new feature class
arcpy.CopyFeatures_management(output_feature_layer, output_feature_class)
print("Saved the selected features to a new feature class:", output_feature_class)


In [ ]:
#Then I will go through and conduct my interpolations
#First is IDW
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\Lab 3 v2\Lab 3 v2.gdb"):
    Idw_elev = arcpy.sa.Idw(
        in_point_features="Sampled_Elevation_Data",
        z_field="grid_code",
        cell_size=2160,
        power=2,
        search_radius="VARIABLE 12",
        in_barrier_polyline_features=None
    )
    Idw_elev.save(r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\Lab 3 v2\Lab 3 v2.gdb\Idw_Elev")

In [ ]:
#Then splining
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\Lab 3 v2\Lab 3 v2.gdb"):
    Spline_elev = arcpy.sa.Spline(
        in_point_features="Sampled_Elevation_Data",
        z_field="grid_code",
        cell_size=2160,
        spline_type="REGULARIZED",
        weight=0.1,
        number_points=12
    )
    Spline_elev.save(r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\Lab 3 v2\Lab 3 v2.gdb\Spline_elev")

In [ ]:
#And then kriging
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\Lab 3 v2\Lab 3 v2.gdb"):
    Kriging_elev = arcpy.sa.Kriging(
        in_point_features="Sampled_Elevation_Data",
        z_field="grid_code",
        kriging_model="Spherical 2160.000000 # # #",
        cell_size=2160,
        search_radius="VARIABLE 12",
        out_variance_prediction_raster=None
    )
    Kriging_elev.save(r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\Lab 3 v2\Lab 3 v2.gdb\Kriging_elev")

In [ ]:
#The interpolations need to be saved in our PostGIS database. As they are rasters, I will first resample them to cut down on file size

arcpy.management.Resample(
    in_raster="Idw_elev",
    out_raster=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\Lab 3 v2\Lab 3 v2.gdb\Idw_Elev_Resample",
    cell_size="0.1 0.1",
    resampling_type="NEAREST"
)
arcpy.management.Resample(
    in_raster="Kriging_elev",
    out_raster=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\Lab 3 v2\Lab 3 v2.gdb\Kriging_Elev_Resample",
    cell_size="0.1 0.1",
    resampling_type="NEAREST"
)
arcpy.management.Resample(
    in_raster="Spline_elev",
    out_raster=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\Lab 3 v2\Lab 3 v2.gdb\Spline_Elev_Resample",
    cell_size="0.1 0.1",
    resampling_type="NEAREST"
)
print("Resampling complete")

In [13]:
# Then we will convert these rasters to points for easier storage
arcpy.conversion.RasterToPoint(
    in_raster="Spline_Elev_Resample",
    out_point_features=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\Lab 3 v2\Lab 3 v2.gdb\Spline_Elev_Point",
    raster_field="Value"
)
arcpy.conversion.RasterToPoint(
    in_raster="Kriging_Elev_Resample",
    out_point_features=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\Lab 3 v2\Lab 3 v2.gdb\Kriging_Elev_Point",
    raster_field="Value"
)
arcpy.conversion.RasterToPoint(
    in_raster="Idw_Elev_Resample",
    out_point_features=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\Lab 3 v2\Lab 3 v2.gdb\Idw_Elev_Point",
    raster_field="Value"
)
print("Raster to Point complete")

<Result 'C:\\Users\\conno\\OneDrive\\Documents\\ArcGIS\\Projects\\Lab 3 v2\\Lab 3 v2.gdb\\Idw_Elev_Point'>

In [3]:
#We will then save these layers to our PostGIS database
arcpy.conversion.ExportFeatures('Kriging_Elev_Point',r'C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\Lab 3 v2\PostgreSQL-34-lab3(postgres).sde\lab3.postgres.Kriging_elev_point')

arcpy.conversion.ExportFeatures('Idw_Elev_Point',r'C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\Lab 3 v2\PostgreSQL-34-lab3(postgres).sde\lab3.postgres.Idw_elev_point')

arcpy.conversion.ExportFeatures('Spline_Elev_Point',r'C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\Lab 3 v2\PostgreSQL-34-lab3(postgres).sde\lab3.postgres.Spline_elev_point')


<Result 'C:\\Users\\conno\\OneDrive\\Documents\\ArcGIS\\Projects\\Lab 3 v2\\PostgreSQL-34-lab3(postgres).sde\\lab3.postgres.Kriging_elev_point'>

### DEM Comparison

In [19]:
# Then we will perform our interpolation accuracy assessment
# To begin, we will use the rendomly generated points to sample our interpolations
arcpy.sa.Sample(
    in_rasters="Idw_elev",
    in_location_data="Random_Selected_Elevation_Features",
    out_table=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\Lab 3 v2\Lab 3 v2.gdb\Sample_Idw_Elev",
    resampling_type="NEAREST",
    unique_id_field="OBJECTID",
    process_as_multidimensional="CURRENT_SLICE",
    acquisition_definition=None,
    statistics_type="",
    percentile_value=None,
    buffer_distance=None,
    layout="ROW_WISE",
    generate_feature_class="TABLE"
)
print("Sampling complete")

Sampling complete


In [20]:
arcpy.sa.Sample(
    in_rasters="Spline_elev",
    in_location_data="Random_Selected_Elevation_Features",
    out_table=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\Lab 3 v2\Lab 3 v2.gdb\Sample_Spline_Elev",
    resampling_type="NEAREST",
    unique_id_field="OBJECTID",
    process_as_multidimensional="CURRENT_SLICE",
    acquisition_definition=None,
    statistics_type="",
    percentile_value=None,
    buffer_distance=None,
    layout="ROW_WISE",
    generate_feature_class="TABLE"
)
print("Sampling complete")

Sampling complete


In [21]:
arcpy.sa.Sample(
    in_rasters="Kriging_elev",
    in_location_data="Random_Selected_Elevation_Features",
    out_table=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\Lab 3 v2\Lab 3 v2.gdb\Sample_Kriging_Elev",
    resampling_type="NEAREST",
    unique_id_field="OBJECTID",
    process_as_multidimensional="CURRENT_SLICE",
    acquisition_definition=None,
    statistics_type="",
    percentile_value=None,
    buffer_distance=None,
    layout="ROW_WISE",
    generate_feature_class="TABLE"
)
print("Sampling complete")

Sampling complete


In [22]:
# Define the input feature class
input_feature_class = "Random_Selected_Elevation_Features"

# Convert feature class to NumPy array
fields = ['OID@', "grid_code", "SHAPE@X", "SHAPE@Y"]
array = arcpy.da.FeatureClassToNumPyArray(input_feature_class, fields)

# Convert NumPy array to DataFrame
df_original = pd.DataFrame(array)

# Rename the OID@ field to ObjectID
df_original.rename(columns={"OID@": "OBJECTID"}, inplace=True)

# Print the DataFrame
print(df_original)


    OBJECTID  grid_code     SHAPE@X     SHAPE@Y
0          4     1060.0  357275.332  5444935.37
1         16     1065.0  352275.332  5424935.37
2         37      980.0  232275.332  5394935.37
3         53     1268.0  347275.332  5379935.37
4         69     1055.0  242275.332  5364935.37
..       ...        ...         ...         ...
95       908     1096.0  532275.332  4869935.37
96       915     1283.0  492275.332  4864935.37
97       918     1197.0  552275.332  4864935.37
98       940     1099.0  572275.332  4854935.37
99       950     1642.0  227275.332  4844935.37

[100 rows x 4 columns]


In [ ]:
# We will then create individual dataframes for the interpolations
table_paths = [
    "Sample_Idw_Elev",
    "Sample_Kriging_Elev",
    "Sample_Spline_Elev"
]

dfs = []

for table_path in table_paths:
    array = arcpy.da.TableToNumPyArray(table_path, "*")
    df = pd.DataFrame(array)
    dfs.append(df)

for i, df in enumerate(dfs):
    print(f"DataFrame {i+1} ({table_paths[i]}):\n{df}\n")

In [ ]:
# We will then join the dataframes
dfs_to_join = [
    df_original,  # Temperature_Data DataFrame
    dfs[0],  # Sample_Idw_Temp DataFrame
    dfs[1],  # Sample_Kriging_Temp DataFrame
    dfs[2]   # Sample_Spline_Temp DataFrame
]

suffixes = ['_original', '_idw', '_kriging', '_spline']
for i, df_to_join in enumerate(dfs_to_join[1:], start=1):
    df_original = df_original.merge(df_to_join, how='left', left_on='OBJECTID', right_on='OBJECTID', suffixes=('', suffixes[i]))

# Print the joined DataFrame
print(df_original)


In [ ]:
df_original['Idw_Elev_Band_1_difference'] = df_original['grid_code'] - df_original['Idw_Elev_Band_1']
df_original['Spline_Elev_Band_1_difference'] = df_original['grid_code'] - df_original['Spline_elev_Band_1']
df_original['Kriging_Elev_Band_1_difference'] = df_original['grid_code'] - df_original['Kriging_elev_Band_1']

df_original

In [ ]:
# Then we calculate mean for each method
idw_mean = df_original['Idw_Elev_Band_1_difference'].mean()
spline_mean = df_original['Spline_Elev_Band_1_difference'].mean()
kriging_mean = df_original['Kriging_Elev_Band_1_difference'].mean()

# Then use that to caluclate RMSE for each method
idw_rmse = np.sqrt(np.mean(df_original['Idw_Elev_Band_1_difference']**2))
spline_rmse = np.sqrt(np.mean(df_original['Spline_Elev_Band_1_difference']**2))
kriging_rmse = np.sqrt(np.mean(df_original['Kriging_Elev_Band_1_difference']**2))

# Then we create a dictionary to hold the results
accuracy_results = {
    'Method': ['IDW', 'Spline', 'Kriging'],
    'Mean': [idw_mean, spline_mean, kriging_mean],
    'RMSE': [idw_rmse, spline_rmse, kriging_rmse]
}

# We save the data into a dataframe from the dictionary
accuracy_df = pd.DataFrame(accuracy_results)

print(accuracy_df)

In [27]:
# Then we define the path to save the table
output_path = r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\Lab 3 v2\accuracy_elevation.csv"

# Finally we save the accuracy dataframe to a CSV file
accuracy_df.to_csv(output_path, index=False)

print("Accuracy DataFrame saved to:", output_path)

Accuracy DataFrame saved to: C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\Lab 3 v2\accuracy_elevation.csv


In [28]:
#Then we save the CSV to our database
sde_connection = r'C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\Lab 3 v2\PostgreSQL-34-lab3(postgres).sde'
output_table = "accuracy_elevation"
csv_file = r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\Lab 3 v2\accuracy_elevation.csv"
arcpy.TableToTable_conversion(csv_file, sde_connection, output_table)

print(f"Table '{output_table}' saved to SDE connection '{sde_connection}'.")

Table 'accuracy_elevation' saved to SDE connection 'C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\Lab 3 v2\PostgreSQL-34-lab3(postgres).sde'.


In [ ]:
# Now we will convert the sampling differences into a point feature
# Let us start with IDW
data_array = df_original[['SHAPE@X', 'SHAPE@Y', 'Idw_Elev_Band_1_difference']].to_numpy()
spatial_reference = arcpy.SpatialReference(4326)  # WGS 1984
feature_class_name = "Idw_Difference_Point_Elev"
arcpy.management.CreateFeatureclass(arcpy.env.workspace, feature_class_name, "POINT", spatial_reference=spatial_reference)
arcpy.management.AddField(feature_class_name, "Idw_Elev_Band_1_difference", "DOUBLE")
with arcpy.da.InsertCursor(feature_class_name, ['SHAPE@X', 'SHAPE@Y', 'Idw_Temper_Band_1_difference']) as cursor:
    for row in data_array:
        cursor.insertRow(row)

print("Point feature class created successfully.")

In [ ]:
# Then Kriging
data_array = df_original[['SHAPE@X', 'SHAPE@Y', 'Kriging_Elev_Band_1_difference']].to_numpy()
spatial_reference = arcpy.SpatialReference(4326)
feature_class_name = "Kriging_Difference_Point_Elev"
arcpy.management.CreateFeatureclass(arcpy.env.workspace, feature_class_name, "POINT", spatial_reference=spatial_reference)
arcpy.management.AddField(feature_class_name, "Kriging_Elev_Band_1_difference", "DOUBLE")
with arcpy.da.InsertCursor(feature_class_name, ['SHAPE@X', 'SHAPE@Y', 'Kriging_Temper_Band_1_difference']) as cursor:
    for row in data_array:
        cursor.insertRow(row)

print("Point feature class created successfully.")

In [ ]:
# And then splining
data_array = df_original[['SHAPE@X', 'SHAPE@Y', 'Spline_Elev_Band_1_difference']].to_numpy()
spatial_reference = arcpy.SpatialReference(4326)  # WGS 1984
feature_class_name = "Spline_Difference_Point_Elev"
arcpy.management.CreateFeatureclass(arcpy.env.workspace, feature_class_name, "POINT", spatial_reference=spatial_reference)
arcpy.management.AddField(feature_class_name, "Spline_Elev_Band_1_difference", "DOUBLE")
with arcpy.da.InsertCursor(feature_class_name, ['SHAPE@X', 'SHAPE@Y', 'Spline_Temper_Band_1_difference']) as cursor:
    for row in data_array:
        cursor.insertRow(row)

print("Point feature class created successfully.")

In [30]:
#Finally, we will save the Kriging difference point layer to our PostGIS database
arcpy.conversion.ExportFeatures('Kriging_Difference_Point_Elev',r'C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\Lab 3 v2\PostgreSQL-34-lab3(postgres).sde\lab3.postgres.Kriging_Differ_Elev')


<Result 'C:\\Users\\conno\\OneDrive\\Documents\\ArcGIS\\Projects\\Lab 3 v2\\PostgreSQL-34-lab3(postgres).sde\\lab3.postgres.Kriging_Differ_Elev'>